In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import wrangle

import warnings
warnings.filterwarnings("ignore")

np.set_printoptions(suppress=True)

SyntaxError: invalid syntax (wrangle.py, line 7)

In [ ]:
df = wrangle.acquire_game_sales()

In [ ]:
df.head()

In [ ]:
df = wrangle.clean_game_sales(df)

In [ ]:
df.head()

In [ ]:
train, validate, test = wrangle.split_game_sales(df)

In [ ]:
X_train, X_validate, X_test, y_train, y_validate, y_test = wrangle.split_train_validate_test(train, validate, test)

In [ ]:
train_scaled, validate_scaled, test_scaled = wrangle.scale_my_data(train, validate, test)